In [1]:
from requests import Session
import pandas as pd
import time
import hashlib
import hmac
import base64
import json

# Import Classified

In [2]:
import sys
sys.path.append('..')
import classified

# GLOBAL

In [4]:
api_passphrase = classified.SPOT_PASSPHRASE
api_key = classified.SPOT_KEY
api_secret = classified.SPOT_SECRET

In [5]:
def authentication(method, endpoint, data_json=""):
    now = int(time.time() * 1000)
    str_to_sign = str(now) + method.upper() + endpoint + data_json
    signature = base64.b64encode(
        hmac.new(api_secret.encode('utf-8'), str_to_sign.encode('utf-8'), hashlib.sha256).digest())
    passphrase = base64.b64encode(hmac.new(api_secret.encode('utf-8'), api_passphrase.encode('utf-8'), hashlib.sha256).digest())
    headers = {
        "KC-API-SIGN": signature,
        "KC-API-TIMESTAMP": str(now),
        "KC-API-KEY": api_key,
        "KC-API-PASSPHRASE": passphrase,
        "KC-API-KEY-VERSION": '2',
        "Content-Type": "application/json"
    }
    return headers

# Private API

In [6]:
class PrivateData:
    # https://docs.kucoin.com/ - API Documentaion

    def __init__(self):
        self.apiurl = "https://api.kucoin.com"  # base url for kucoin
        self.session = Session()

    # returns account balance
    def getBalance(self):
        url = self.apiurl + '/api/v1/accounts'
        headers = authentication('get', '/api/v1/accounts')
        r = self.session.get(url, headers=headers)
        data = r.json()["data"]
        df = pd.DataFrame.from_dict(data)
        return df
    
    # returns the full order book for a trading pair
    def getFullOrderBook(self, symbol):  # FIXME
        url = self.apiurl + '/api/v3/market/orderbook/level2'
        headers = authentication(method='get', endpoint='/api/v3/market/orderbook/level2')
        parameters = {"symbol": symbol}
        r = self.session.get(url, headers=headers, params=parameters)
        data = r.json()
        return data
    
    # places a new order
    def placeOrder(self, side, symbol, price, size):
        url = self.apiurl + '/api/v1/orders'
        data = {"clientOid": "01", "side": side, "symbol": symbol, "price": price,"size": size}
        data_json = json.dumps(data)
        headers = authentication('post', '/api/v1/orders', data_json=data_json)
        response = self.session.post(url, headers=headers, data=data_json)
        return response.json()


In [7]:
kucoinAPI = PrivateData()

In [8]:
kucoinAPI.getBalance()

,id,currency,type,balance,available,holds
0,610e0377e962a1000673e64b,USDT,trade,0.00000081,0.00000081,0
1,610e0477e962a10006754f94,ETH,trade,0.00000007,0.00000007,0
2,617ccb73c429720001f948f1,ADA,main,0,0,0
3,617ccb9c21d14c0001a1a233,ADA,trade,0,0,0
4,6244482fd126950001728bf2,ALBT,trade,0,0,0
5,62335013e8e40d000104c6f8,APE,trade,0,0,0
6,617af44672346c0001579056,BNB,main,0,0,0
7,617af41e7a7722000134b0ad,BNB,trade,0,0,0
8,62bdce0407ced90001196bb5,BTC,trade,0,0,0
9,6243a55ad7856d00012e4ded,CGG,trade,0,0,0


In [49]:
kucoinAPI.placeOrder("buy", "ETH-USDT", "800", "0.01")

{'code': '400100', 'msg': 'validation.createOrder.clientOidDuplicated'}